In [3]:
import numpy as np
import csv

In [21]:
RND_MEAN = 0.0
RND_STD = 1.0
LEARNING_RATE = 0.001

In [9]:
# def overflowed_sigmoid(x):
#     return 1.0 / (1.0 + np.exp(-x))

# print(overflowed_sigmoid(1.0))
# print(overflowed_sigmoid(-1000.0))

0.7310585786300049
0.0


/var/folders/ty/sj9j6sl158j5bv97y8pxd76c0000gn/T/ipykernel_29210/2617290605.py:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


In [13]:
def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))

def relu(x):
    return np.maximum(x, 0)

def sigmoid_derv(x, y):
    #y = sigmoid(x)
    #dy/dx = sigmod(x) (1 - sigmoid(x))
    return y * (1 - y)

In [22]:
def sigmoid_cross_entropy_with_logits(z, x):
    # H = x - zx + log(1 + exp(-x))
    # H = max(x, 0) - zx + log(1 + exp(-abs(x)))
    return relu(x) - z * x + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    # H = x - zx + log( 1 + exp(-x) )
    # dH / dx = 1 - z + (-exp(-x) / 1 + exp(-x))
    # dH / dx = 1 - z - 1 + ( 1 / 1 + exp(-x))
    # dH / dx = -z + sigmoid(x)
    return -z + sigmoid(x)

In [25]:
def eval_accuracy(output, y):
    est_yes = np.greater(output, 0) # 추정 값들 중 참인거
    ans_yes = np.greater(y, 0.5) # 실제 값들 중 참인거

    est_no = np.logical_not(est_yes) # 추정 값들 중 거짓
    ans_no = np.logical_not(ans_yes) # 실제 값들 중 거짓

    tp = np.sum(np.logical_and(est_yes, ans_yes)) # 답은 o 인데 추정은 o
    fp = np.sum(np.logical_and(est_yes, ans_no)) # 답은 x 인데 추정은 o 인거
    fn = np.sum(np.logical_and(est_no, ans_no)) # 답은 x  인데 추정은 x
    tn = np.sum(np.logical_and(est_no, ans_yes)) # 담은 o 인데 추정은 x
    
    accuracy = safe_div(tp+tn, tp+tn+fp+fn)
    precistion = safe_div(tp, tp+fp)
    recall = safe_div(tp, tp+tn)
    f1 = 2 * safe_div(recall*precistion, recall+precistion)
    
    return [accuracy, precistion, recall, f1]

def safe_div(p, q):
    p, q = float(p), float(q)
    if np.abs(q) < 1e-20: return np.sign(p)
    return p / q

In [29]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

def forward_postproc(output, y):
    entropy = sigmoid_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy)
    return loss, [y, output, entropy]

def backprop_neuralnet(G_upstream, x):
    global weight, bias
    g_output_w = x.transpose()

    G_w = np.matmul(g_output_w, G_upstream)
    G_b = np.sum(G_upstream, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux

    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    return G_output


In [30]:
def run_train(x, y):

    #wx + b 계산
    output, aux_nn = forward_neuralnet(x)
    #(wx + b - y) ^ 2 계산
    loss, aux_pp = forward_postproc(output, y)
    # wx + b 와 y 간의
    accuracy = eval_accuracy(output, y)
    
    #dL/dL
    dL_dL = 1.0
    G_output = backprop_postproc(dL_dL, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [1]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_dim
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_dim], test_data[:, -output_dim:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_dim
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_dim], train_data[:, -output_dim:]

In [32]:
def train_and_test(epochs, batch_size, report):
    step_count = arrange_data(batch_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epochs):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(batch_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            acc_str = ','.join(['%5.3f']*4) % tuple(acc)
            print('Epoch {}: loss={:5.3f}, accuracy={}'. \
                  format(epoch+1, np.mean(losses), acc_str))
            
    final_acc = run_test(test_x, test_y)
    acc_str = ','.join(['%5.3f']*4) % tuple(final_acc)
    print('\nFinal Test: final accuracy = {}'.format(acc_str))

In [33]:
def init_model():
    global weight, bias, input_dim, output_dim
    weight = np.random.normal(RND_MEAN, RND_STD, [input_dim, output_dim])
    bias = np.zeros(output_dim)

In [2]:
def load_pulsar_dataset(adjust_ratio):
    pulsars, stars = [], []

    global data, test_data, input_dim, output_dim
    #rows = []
    with open("dataset/pulsar_data_train.csv", 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        for row in csvreader:
            for i in range(len(row)):
                if row[i] == '':
                    row[i] = 0.0
                
            if row[8] == '1': pulsars.append(row)
            else: stars.append(row)
            #rows.append(row)
    
    input_dim, output_dim = 8, 1

    star_cnt, pulsar_cnt = len(stars), len(pulsars)

    if adjust_ratio:
        data = np.zeros([2*star_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        for n in range(star_cnt):
            data[star_cnt+n] = np.asarray(pulsars[n % pulsar_cnt], dtype=np.float32)
    else:
        data = np.zeros([star_cnt+pulsar_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype=np.float32)
        data[star_cnt:, :] = np.asarray(pulsars, dtype=np.float32)



    rows = []
    with open("dataset/pulsar_data_test.csv", 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvfile, None)
        for row in csvreader:
            for i in range(len(row)):
                if row[i] == '':
                    row[i] = 0.0


            rows.append(row)

    test_data = np.asarray(rows, dtype=np.float32)


In [4]:
def pulsar_exec(epochs=10, batch_size=10, report=1, adjust_ratio=False):
    load_pulsar_dataset(adjust_ratio)
    init_model()
    train_and_test(epochs, batch_size, report)